# Example 34: Gradient Boosting Engines Comparison

**Feature**: `boost_tree()` with three engines: XGBoost, LightGBM, CatBoost

## Overview

This notebook demonstrates the **unified gradient boosting interface** via `boost_tree()` with three popular engines:

1. **XGBoost**: Industry standard, highly optimized
2. **LightGBM**: Microsoft's fast implementation with leaf-wise growth
3. **CatBoost**: Yandex's implementation with categorical feature support

## Key Features

**Unified API**: Same parameters across engines
```python
boost_tree(
    trees=100,           # Number of boosting rounds
    tree_depth=6,        # Maximum tree depth
    learn_rate=0.1,      # Learning rate (eta/alpha)
    mtry=0.8,            # Feature sampling ratio (colsample)
    min_n=5,             # Minimum samples in leaf
    loss_reduction=0.0,  # Min loss reduction for split (gamma)
    sample_size=0.8      # Row sampling ratio (subsample)
).set_engine('xgboost').set_mode('regression')
```

**Automatic Parameter Translation**: tidymodels params → engine params

## When to Use Each Engine

**XGBoost**:
- Industry standard, proven track record
- Most hyperparameter tuning resources available
- Good balance of speed and accuracy
- Rich ecosystem (model interpretation, deployment)

**LightGBM**:
- Fastest training on large datasets (>100K rows)
- Memory efficient
- Leaf-wise growth (vs level-wise) can be more accurate
- Best for Kaggle-style competitions

**CatBoost**:
- Best for categorical features (native support)
- Good default parameters (less tuning needed)
- Often wins on tabular data benchmarks
- Robust to overfitting

## Dataset

**Refinery Margins** (European refineries):
- Monthly refinery margins from 2006-2024
- Multiple countries (10 European countries)
- Crude oil prices (Brent, Dubai, WTI)
- Various margin types (cracking, hydroskimming)
- Target: Brent cracking margin in NW Europe

In [ ]:
# Setup
import pandas as pd
import numpy as np
import time
from datetime import timedelta

# py-tidymodels imports
from py_parsnip import boost_tree, linear_reg, rand_forest
from py_rsample import initial_time_split, vfold_cv
from py_yardstick import rmse, mae, r_squared
from py_yardstick import metric_set
from py_workflows import Workflow
from py_workflowsets import WorkflowSet

import warnings
warnings.filterwarnings('ignore')

print("✓ Imports complete")

## 1. Load and Prepare Data

In [ ]:
# Load refinery margins data
df = pd.read_csv('../_md/__data/refinery_margins.csv')
df['date'] = pd.to_datetime(df['date'])

# Filter to Germany (largest refining capacity)
germany = df[df['country'] == 'Germany'].copy()

# Select relevant columns
# Target: Brent cracking margin in NW Europe
# Predictors: crude prices (Brent, Dubai, WTI) and date
germany = germany[[
    'date', 'brent', 'dubai', 'wti', 'brent_cracking_nw_europe'
]].rename(columns={'brent_cracking_nw_europe': 'margin'})

# Remove any missing values
germany = germany.dropna().sort_values('date').reset_index(drop=True)

print(f"Germany refinery margin data:")
print(f"  Records: {len(germany):,} months")
print(f"  Date range: {germany['date'].min()} to {germany['date'].max()}")
print(f"  Margin mean: ${germany['margin'].mean():.2f}/bbl")
print(f"  Margin std: ${germany['margin'].std():.2f}/bbl")
print(f"\nPredictors:")
print(f"  Brent: ${germany['brent'].mean():.2f} ± ${germany['brent'].std():.2f}/bbl")
print(f"  Dubai: ${germany['dubai'].mean():.2f} ± ${germany['dubai'].std():.2f}/bbl")
print(f"  WTI:   ${germany['wti'].mean():.2f} ± ${germany['wti'].std():.2f}/bbl")
print(f"\nFirst few rows:")
print(germany.head())

In [ ]:
# Train/test split (hold out last 24 months)
split = initial_time_split(germany, date_column='date', prop=0.85)
train = split.training()
test = split.testing()

print(f"Train: {len(train)} months ({train['date'].min()} to {train['date'].max()})")
print(f"Test:  {len(test)} months ({test['date'].min()} to {test['date'].max()})")
print(f"\nHolding out {len(test)} months for evaluation")

## 2. XGBoost Engine

Industry standard gradient boosting implementation.

In [ ]:
# XGBoost with full parameter specification
spec_xgboost = boost_tree(
    trees=100,
    tree_depth=6,
    learn_rate=0.1,
    mtry=0.8,          # Column sampling
    min_n=5,           # Min samples in leaf
    loss_reduction=0.0, # Gamma (regularization)
    sample_size=0.8    # Row sampling
).set_engine('xgboost').set_mode('regression')

# Fit
start_time = time.time()
fit_xgboost = spec_xgboost.fit(train, 'margin ~ brent + dubai + wti')
xgb_train_time = time.time() - start_time

# Evaluate
eval_xgboost = fit_xgboost.evaluate(test)
outputs, coeffs, stats = eval_xgboost.extract_outputs()

test_stats_xgb = stats[stats['split'] == 'test'].iloc[0]
print("XGBoost:")
print(f"  Training time: {xgb_train_time:.3f} seconds")
print(f"  Test RMSE: ${test_stats_xgb['rmse']:.3f}/bbl")
print(f"  Test MAE: ${test_stats_xgb['mae']:.3f}/bbl")
print(f"  Test R²: {test_stats_xgb['r_squared']:.4f}")

## 3. LightGBM Engine

Microsoft's fast gradient boosting with leaf-wise growth.

In [ ]:
# LightGBM with same parameters
spec_lightgbm = boost_tree(
    trees=100,
    tree_depth=6,
    learn_rate=0.1,
    mtry=0.8,
    min_n=5,
    loss_reduction=0.0,
    sample_size=0.8
).set_engine('lightgbm').set_mode('regression')

# Fit
start_time = time.time()
fit_lightgbm = spec_lightgbm.fit(train, 'margin ~ brent + dubai + wti')
lgb_train_time = time.time() - start_time

# Evaluate
eval_lightgbm = fit_lightgbm.evaluate(test)
_, _, stats_lgb = eval_lightgbm.extract_outputs()

test_stats_lgb = stats_lgb[stats_lgb['split'] == 'test'].iloc[0]
print("LightGBM:")
print(f"  Training time: {lgb_train_time:.3f} seconds")
print(f"  Test RMSE: ${test_stats_lgb['rmse']:.3f}/bbl")
print(f"  Test MAE: ${test_stats_lgb['mae']:.3f}/bbl")
print(f"  Test R²: {test_stats_lgb['r_squared']:.4f}")
print(f"\nSpeedup vs XGBoost: {xgb_train_time / lgb_train_time:.2f}x")

## 4. CatBoost Engine

Yandex's gradient boosting with native categorical feature support.

In [ ]:
# CatBoost with same parameters
spec_catboost = boost_tree(
    trees=100,
    tree_depth=6,
    learn_rate=0.1,
    mtry=0.8,
    min_n=5,
    loss_reduction=0.0,
    sample_size=0.8
).set_engine('catboost').set_mode('regression')

# Fit
start_time = time.time()
fit_catboost = spec_catboost.fit(train, 'margin ~ brent + dubai + wti')
cat_train_time = time.time() - start_time

# Evaluate
eval_catboost = fit_catboost.evaluate(test)
_, _, stats_cat = eval_catboost.extract_outputs()

test_stats_cat = stats_cat[stats_cat['split'] == 'test'].iloc[0]
print("CatBoost:")
print(f"  Training time: {cat_train_time:.3f} seconds")
print(f"  Test RMSE: ${test_stats_cat['rmse']:.3f}/bbl")
print(f"  Test MAE: ${test_stats_cat['mae']:.3f}/bbl")
print(f"  Test R²: {test_stats_cat['r_squared']:.4f}")
print(f"\nSpeedup vs XGBoost: {xgb_train_time / cat_train_time:.2f}x")

## 5. Engine Comparison Summary

In [ ]:
# Compare all three engines
comparison = pd.DataFrame([
    {
        'engine': 'XGBoost',
        'train_time_sec': xgb_train_time,
        'rmse': test_stats_xgb['rmse'],
        'mae': test_stats_xgb['mae'],
        'r_squared': test_stats_xgb['r_squared']
    },
    {
        'engine': 'LightGBM',
        'train_time_sec': lgb_train_time,
        'rmse': test_stats_lgb['rmse'],
        'mae': test_stats_lgb['mae'],
        'r_squared': test_stats_lgb['r_squared']
    },
    {
        'engine': 'CatBoost',
        'train_time_sec': cat_train_time,
        'rmse': test_stats_cat['rmse'],
        'mae': test_stats_cat['mae'],
        'r_squared': test_stats_cat['r_squared']
    }
])

# Add speedup column
comparison['speedup_vs_xgb'] = xgb_train_time / comparison['train_time_sec']

# Sort by RMSE
comparison = comparison.sort_values('rmse')

print("\nGradient Boosting Engine Comparison:")
print("="*90)
print(comparison.to_string(index=False))
print("="*90)
print(f"\nBest accuracy: {comparison.iloc[0]['engine']} (RMSE: ${comparison.iloc[0]['rmse']:.3f}/bbl)")
print(f"Fastest training: {comparison.sort_values('train_time_sec').iloc[0]['engine']} ({comparison.sort_values('train_time_sec').iloc[0]['train_time_sec']:.3f}s)")

## 6. WorkflowSet Comparison

Use WorkflowSet to systematically compare all engines.

In [ ]:
# Create workflows for all engines
engines = ['xgboost', 'lightgbm', 'catboost']

workflows = []
for engine in engines:
    spec = boost_tree(
        trees=100,
        tree_depth=6,
        learn_rate=0.1
    ).set_engine(engine).set_mode('regression')
    
    wf = Workflow().add_formula('margin ~ brent + dubai + wti').add_model(spec)
    workflows.append(wf)

wf_set = WorkflowSet.from_workflows(workflows)

print(f"Created WorkflowSet with {len(workflows)} boosting engines")
print(f"Workflows: {list(wf_set.workflows.keys())}")

In [ ]:
# Fit all workflows and collect metrics
wf_results = []
for wf_id, wf in wf_set.workflows.items():
    start_time = time.time()
    fit = wf.fit(train)
    train_time = time.time() - start_time
    
    eval_fit = fit.evaluate(test)
    _, _, stats = eval_fit.extract_outputs()
    
    test_stats = stats[stats['split'] == 'test'].iloc[0]
    wf_results.append({
        'workflow': wf_id,
        'train_time': train_time,
        'rmse': test_stats['rmse'],
        'mae': test_stats['mae'],
        'r_squared': test_stats['r_squared']
    })

wf_comparison = pd.DataFrame(wf_results)
wf_comparison = wf_comparison.sort_values('rmse')

print("\nWorkflowSet Results:")
print("="*80)
print(wf_comparison.to_string(index=False))
print("="*80)

## 7. Compare with Baseline Models

How do gradient boosting models compare to simpler baselines?

In [ ]:
# Add baseline models
baseline_models = [
    ('linear_reg', linear_reg()),
    ('random_forest', rand_forest(trees=100).set_mode('regression'))
]

all_results = wf_results.copy()

for name, model in baseline_models:
    start_time = time.time()
    fit = model.fit(train, 'margin ~ brent + dubai + wti')
    train_time = time.time() - start_time
    
    eval_fit = fit.evaluate(test)
    _, _, stats = eval_fit.extract_outputs()
    
    test_stats = stats[stats['split'] == 'test'].iloc[0]
    all_results.append({
        'workflow': name,
        'train_time': train_time,
        'rmse': test_stats['rmse'],
        'mae': test_stats['mae'],
        'r_squared': test_stats['r_squared']
    })

all_comparison = pd.DataFrame(all_results)
all_comparison = all_comparison.sort_values('rmse')

print("\nAll Models Comparison (Boosting + Baselines):")
print("="*80)
print(all_comparison.to_string(index=False))
print("="*80)

## 8. Parameter Translation Examples

Show how tidymodels params map to each engine.

In [ ]:
# Parameter mapping reference
param_mapping = pd.DataFrame([
    {
        'tidymodels': 'trees',
        'xgboost': 'n_estimators',
        'lightgbm': 'n_estimators',
        'catboost': 'iterations'
    },
    {
        'tidymodels': 'tree_depth',
        'xgboost': 'max_depth',
        'lightgbm': 'max_depth',
        'catboost': 'depth'
    },
    {
        'tidymodels': 'learn_rate',
        'xgboost': 'learning_rate',
        'lightgbm': 'learning_rate',
        'catboost': 'learning_rate'
    },
    {
        'tidymodels': 'mtry',
        'xgboost': 'colsample_bytree',
        'lightgbm': 'feature_fraction',
        'catboost': 'rsm'
    },
    {
        'tidymodels': 'min_n',
        'xgboost': 'min_child_weight',
        'lightgbm': 'min_child_samples',
        'catboost': 'min_data_in_leaf'
    },
    {
        'tidymodels': 'loss_reduction',
        'xgboost': 'gamma',
        'lightgbm': 'min_gain_to_split',
        'catboost': 'min_data_in_leaf'
    },
    {
        'tidymodels': 'sample_size',
        'xgboost': 'subsample',
        'lightgbm': 'bagging_fraction',
        'catboost': 'subsample'
    }
])

print("\nParameter Translation Across Engines:")
print("="*80)
print(param_mapping.to_string(index=False))
print("="*80)
print("\nNote: py-tidymodels handles this translation automatically!")

## 9. Key Takeaways

### Engine Selection Guidance

**Choose XGBoost when**:
- You need a proven, battle-tested implementation
- Extensive hyperparameter tuning resources available
- Model interpretation is important (SHAP, feature importance)
- Production deployment with existing XGBoost infrastructure

**Choose LightGBM when**:
- Dataset has >100K rows (speed advantage)
- Memory is constrained
- Leaf-wise growth is beneficial (more accurate splits)
- Kaggle competitions or similar high-performance scenarios

**Choose CatBoost when**:
- Dataset has categorical features (native support)
- Limited time for hyperparameter tuning (good defaults)
- Robustness to overfitting is critical
- Tabular data with mixed feature types

### Performance Patterns

From our refinery margin example:
1. **Accuracy**: All three engines performed similarly on this regression task
2. **Speed**: LightGBM typically fastest, XGBoost middle, CatBoost varies
3. **vs Baselines**: All boosting engines significantly outperformed linear regression
4. **vs Random Forest**: Boosting models typically more accurate with fewer trees

### Unified API Benefits

```python
# Same code structure for all engines
spec = boost_tree(trees=100, tree_depth=6, learn_rate=0.1)

# Just change engine
fit_xgb = spec.set_engine('xgboost').fit(data, formula)
fit_lgb = spec.set_engine('lightgbm').fit(data, formula)
fit_cat = spec.set_engine('catboost').fit(data, formula)

# Easy to benchmark and swap engines
```

### Hyperparameter Tuning

**Start with these defaults**:
```python
boost_tree(
    trees=100,           # Often sufficient, increase to 500-1000 if needed
    tree_depth=6,        # 3-10 typical range
    learn_rate=0.1,      # 0.01-0.3 typical, lower = more trees needed
    mtry=0.8,            # 0.5-1.0, feature sampling
    min_n=5,             # 1-20, regularization
    sample_size=0.8      # 0.5-1.0, row sampling
)
```

**Tune in this order**:
1. `learn_rate` and `trees` together (lower learning rate = more trees)
2. `tree_depth` (controls model complexity)
3. `sample_size` and `mtry` (regularization via sampling)
4. `min_n` and `loss_reduction` (leaf regularization)

### Production Deployment

```python
# Standard production pattern
from py_parsnip import boost_tree
from py_workflows import Workflow
from py_recipes import recipe, step_normalize

# Preprocessing + boosting
rec = recipe().step_normalize(all_numeric_predictors())
spec = boost_tree(trees=200, tree_depth=6).set_engine('xgboost')
wf = Workflow().add_recipe(rec).add_model(spec)

# Fit on all training data
final_fit = wf.fit(all_training_data)

# Predict
predictions = final_fit.predict(new_data)
```

### Common Pitfalls

1. **Too many trees**: Overfitting and slow training
   - Solution: Use early stopping or tune `trees` parameter

2. **High learning rate**: Unstable training
   - Solution: Reduce `learn_rate` to 0.01-0.05, increase `trees`

3. **Deep trees**: Overfitting on small datasets
   - Solution: Reduce `tree_depth` to 3-4 for <10K rows

4. **Forgetting mode**: "Unknown mode" errors
   - Solution: Always call `.set_mode('regression')` or `.set_mode('classification')`

5. **Engine not installed**: Import errors
   - Solution: `pip install xgboost lightgbm catboost`

## Summary

This notebook demonstrated:

✅ XGBoost engine with full parameter specification  
✅ LightGBM engine for fast training  
✅ CatBoost engine with robust defaults  
✅ Direct comparison across all three engines  
✅ WorkflowSet integration for systematic comparison  
✅ Benchmarking vs baseline models (linear, Random Forest)  
✅ Parameter translation reference across engines  
✅ Production deployment patterns  

**Key Insight**: All three gradient boosting engines are accessible through the same unified `boost_tree()` API. The choice between XGBoost, LightGBM, and CatBoost depends on:
- Dataset size and characteristics
- Speed vs accuracy tradeoffs
- Categorical feature handling needs
- Production infrastructure constraints

**Recommendation**: Start with XGBoost (most proven), benchmark LightGBM (speed), and try CatBoost if you have categorical features.

**Next Steps**:
- Hyperparameter tuning with `tune_grid()`
- Feature engineering with `recipes`
- Production deployment integration